In [31]:
#Importing required libraries 
import numpy as np 
import tensorflow as tf

In [32]:
# Initializing Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

#Reading Data
data = open(r'/content/dataset.txt').read()

#Making sentences Lower case and spliting it 
corpus = data.lower().split("<eos>")
corpus = corpus[:100]

In [33]:
#Cleaning data, removing unwanted symbols.
import re
for i in range(len(corpus)):
 corpus[i] = corpus[i].replace("\n",' ')
 corpus[i] = corpus[i].replace("<eos>",' ')
 corpus[i] = re.sub("[`~!@#$+%*:().'?-].\"", ' ',corpus[i]) 

In [34]:
#Fitting data on tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.utils as ku 
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1 

input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
        
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

predictors, label = input_sequences[:,:-1],input_sequences[:,-1]                                                                               
label = ku.to_categorical(label, num_classes=total_words)        

In [35]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import Regularizer
#Sequential Model 
model = Sequential()

#Adding layers to the model 
model.add(Embedding(total_words,240, input_length=max_sequence_len-1))  
model.add(Bidirectional(LSTM(150))) 
model.add(Dropout(0.2)) 
model.add(Dense(total_words/2, activation='relu'))
model.add(Dense(total_words, activation='softmax')) 

In [36]:
#Compiling Model 
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])  

In [37]:
from keras.callbacks import ModelCheckpoint

# Checkpoint 
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.82):
      model.save('vaibhav.h5')
      print("\nReached 80% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()
#Training model over certain epoches .
model.fit(predictors, label, epochs=1,callbacks = [callbacks], verbose=1)

22/22 [==============================] - 0s 6ms/step - loss: 6.0986 - accuracy: 0.0270


In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:

from tensorflow.keras.models import model_from_json
json_file = open('/content/drive/My Drive/jack1.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights('/content/drive/My Drive/model.h5')
new_model = loaded_model
#Cleaning data, removing unwanted symbols
data = open(r'/content/dataset2.txt').read()
corpus = data.lower().split("<eos>")
import re
for i in range(len(corpus)):
 corpus[i] = corpus[i].replace("\n",' ')
 corpus[i] = corpus[i].replace("<eos>",' ')
 corpus[i] = re.sub("[`~!@#$+%*:().'?-].\"", ' ',corpus[i]) 

sonnet = []  
def predict():
  seed_len = 14
  for _ in range(seed_len): 
    start_idx = np.random.randint(0, len(corpus))
    seed_text = corpus[start_idx]
    for i in range(1):
      token_list = tokenizer.texts_to_sequences([seed_text])[0]
      token_list = pad_sequences([token_list], maxlen= 10, padding='pre')
      predicted = new_model.predict_classes(token_list, verbose=0 )
      output_word = ""
      for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
      seed_text += " " +output_word
    sonnet.append(seed_text)
  return sonnet

d = predict()
for i in d :
  print(i)

 o ’ er all his limbs ambrosial odours shed  but
 but feast their souls on ilion ’ s woes to come  charms
 pale , and unrobed .— faithless ! thou well may'st hide  the
 turn back ! no man her ‘ witched gift withstands  
 for if i die and thou repent t ’ have slain me  not
 but who shall dare , in this refining age  i
 oh , mother nature ! would that i could run  —
 o melancholy soul , whom far and near  leaf
 two happy farmers , learned in love 's lore  
 no rank malaria stains thine atmosphere  the
 a silent heart whose silence loves and longs  your
 as carnal hindrances which held my soul  mantuan
 if ours be not the laurel , ours the palm  seed
 thy eyes like searing irons burn out mine  for
